<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/query_engine/knowledge_graph_rag_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 知识图谱RAG查询引擎

## 图谱RAG

图谱RAG是一种基于知识的RAG方法，用于从给定任务的知识图谱中检索信息。通常，这是为了构建基于与任务相关的实体子图的上下文。

## 基于GraphStore的RAG vs 基于VectorStore的RAG

正如我们在[这个教程](https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KnowledgeGraphIndex_vs_VectorStoreIndex_vs_CustomIndex_combined.html#id1)中比较了Graph RAG在某些用例中的帮助，显示知识图谱作为信息的唯一格式可以缓解由“分割和嵌入”RAG方法的性质引起的几个问题。

## 为什么使用知识图谱RAG查询引擎

在Llama Index中，有两种情景可以应用图谱RAG：

- 使用Llama Index从文档构建知识图谱，使用LLM甚至[本地模型](https://colab.research.google.com/drive/1G6pcR0pXvSkdMQlAK_P-IrYgo-_staxd?usp=sharing)，为此，我们应该选择`KnowledgeGraphIndex`。
- 利用现有的知识图谱，在这种情况下，我们应该使用`KnowledgeGraphRAGQueryEngine`。

> 注意，在Llama Index中与知识图谱相关的第三个查询引擎是`NL2GraphQuery`或`Text2Cypher`，无论是现有的知识图谱还是不是，都可以使用`KnowledgeGraphQueryEngine`来完成。


在开始`Knowledge Graph RAG QueryEngine`演示之前，让我们先为Llama Index的基本准备工作做好准备。


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-llms-azure-openai
%pip install llama-index-graph-stores-nebula
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-azure-openai

In [ ]:
!pip install llama-index

OpenAI 是一个人工智能研究实验室，致力于推动人工智能的发展，以确保它对人类的利益产生积极影响。OpenAI 的使命是确保人工智能技术的发展符合人类的价值观，并能够为全人类带来长期利益。


In [ ]:
# 为OpenAIimport osos.environ["OPENAI_API_KEY"] = "sk-..."import loggingimport syslogging.basicConfig(    stream=sys.stdout, level=logging.INFO)  # logging.DEBUG 用于更详细的输出# 定义LLMfrom llama_index.llms.openai import OpenAIfrom llama_index.core import SettingsSettings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")Settings.chunk_size = 512

Azure是由微软提供的云计算服务，用于构建、测试、部署和管理应用程序和服务。Azure提供了各种服务，包括计算、存储和数据库，可以帮助用户在全球范围内扩展其业务。Azure还提供了人工智能、物联网和区块链等先进的解决方案，以满足不同行业和业务的需求。


In [ ]:
from llama_index.llms.azure_openai import AzureOpenAIfrom llama_index.embeddings.azure_openai import AzureOpenAIEmbedding# 对于Azure OpenAIapi_key = "<api-key>"azure_endpoint = "https://<your-resource-name>.openai.azure.com/"api_version = "2023-07-01-preview"llm = AzureOpenAI(    model="gpt-35-turbo-16k",    deployment_name="my-custom-llm",    api_key=api_key,    azure_endpoint=azure_endpoint,    api_version=api_version,)# 您需要部署自己的嵌入模型以及自己的聊天完成模型embed_model = AzureOpenAIEmbedding(    model="text-embedding-ada-002",    deployment_name="my-custom-embedding",    api_key=api_key,    azure_endpoint=azure_endpoint,    api_version=api_version,)

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

## 准备NebulaGraph

在这个演示中，我们以[NebulaGraphStore](https://gpt-index.readthedocs.io/en/stable/examples/index_structs/knowledge_graph/NebulaGraphKGIndexDemo.html)为例，因此在进行下一步对现有知识图谱执行图RAG之前，让我们确保我们有一个运行中的NebulaGraph，并且已定义了数据模式。

这一步安装了NebulaGraph的客户端，并准备了定义[NebulaGraph图空间](https://docs.nebula-graph.io/3.6.0/1.introduction/2.data-model/)的上下文。


In [ ]:
# 使用以下命令创建一个 NebulaGraph（版本3.5.0或更新版本）集群：# 选项0：适用于使用Docker的机器：`curl -fsSL nebula-up.siwei.io/install.sh | bash`# 选项1：适用于桌面版：NebulaGraph Docker扩展 https://hub.docker.com/extensions/weygu/nebulagraph-dd-ext# 如果不是使用上述方法创建，可以使用以下命令从NebulaGraph的控制台创建：# CREATE SPACE llamaindex(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);# :sleep 10;# USE llamaindex;# CREATE TAG entity(name string);# CREATE EDGE relationship(relationship string);# :sleep 10;# CREATE TAG INDEX entity_index ON entity(name(256));%pip install ipython-ngql nebula3-pythonos.environ["NEBULA_USER"] = "root"os.environ["NEBULA_PASSWORD"] = "nebula"  # 默认为"nebula"os.environ[    "NEBULA_ADDRESS"] = "127.0.0.1:9669"  # 假设NebulaGraph已在本地安装space_name = "llamaindex"edge_types, rel_prop_names = ["relationship"], [    "relationship"]  # 默认值，如果从空的kg创建，则可以省略tags = ["entity"]  # 默认值，如果从空的kg创建，则可以省略

You should consider upgrading via the '/Users/loganmarkewich/llama_index/llama-index/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


然后，我们可以实例化一个 `NebulaGraphStore`，以便将 `StorageContext` 的 `graph_store` 创建为它。


In [ ]:
from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

这里，我们假设使用了与[此教程](https://gpt-index.readthedocs.io/en/latest/examples/query_engine/knowledge_graph_query_engine.html#optional-build-the-knowledge-graph-with-llamaindex)相同的知识图。


## 执行图形RAG查询

最后，让我们演示如何针对现有的知识图执行图形RAG。

我们所需要做的就是使用`RetrieverQueryEngine`，并将其检索器配置为`KnowledgeGraphRAGRetriever`。

`KnowledgeGraphRAGRetriever`执行以下步骤：

- 搜索问题/任务的相关实体
- 从知识图中获取这些实体的子图（默认为2层深度）
- 基于子图构建上下文

请注意，搜索相关实体的方式可以是基于关键词提取或嵌入式的，这由`KnowledgeGraphRAGRetriever`的`retriever_mode`参数控制，支持的选项包括：
- "keyword"
- "embedding"（尚未实现）
- "keyword_embedding"（尚未实现）

以下是如何使用`RetrieverQueryEngine`和`KnowledgeGraphRAGRetriever`的示例：


In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
)

query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever,
)

然后我们可以像这样查询它：


In [ ]:
from IPython.display import display, Markdown

response = query_engine.query(
    "Tell me about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))

Entities processed: ['Star', 'Lord', 'Marvel', 'Quill', 'Galaxy', 'Guardians', 'Guardians of the Galaxy', 'Star-Lord', 'Peter Quill', 'Peter']
Entities processed: ['Star', 'Lord', 'Marvel', 'Quill', 'Galaxy', 'Guardians', 'Guardians of the Galaxy', 'Star-Lord', 'Peter Quill', 'Peter']
Graph RAG context:
The following are knowledge sequence in max depth 2 in the form of `subject predicate, object, predicate_next_hop, object_next_hop ...` extracted based on key entities as subject:
Guardians, is member of, Guardians, was experimented on, by the High Evolutionary
Guardians, is member of, Guardians, considered to tell, origins
Guardians, is member of, Guardians, origins, team-up movie
Guardians, is member of, Guardians, befriended, his fellow Batch 89 test subjects
Guardians, is member of, Guardians, sought to enhance and anthropomorphize animal lifeforms, to create an ideal society
Guardians, is member of, Guardians, is creator of, Rocket
Guardians, is member of, Guardians, is, Mantis
Gua

<b>

Peter Quill is the leader of the Guardians of the Galaxy and the main protagonist of the Guardians of the Galaxy films. He was raised by a group of alien thieves and smugglers, and was abducted from Earth as a child. He is half-human, half-Celestial, and has the ability to wield an energy weapon called the Infinity Stone. He is set to return to the MCU in May 2021.</b>

In [ ]:
response = await query_engine.aquery(
    "Tell me about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=611 request_id=1c07a89e18f19ac7bbc508507c2902d9 response_code=200
Entities processed: ['Star', 'Lord', 'Marvel', 'Quill', 'Galaxy', 'Guardians', 'Guardians of the Galaxy', 'Star-Lord', 'Peter Quill', 'Peter']
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=992 request_id=6517cb63da3364acd33e816a9b3ee242 response_code=200
Entities processed: ['Star', 'Lord', 'Marvel', 'Quill', 'Galaxy', 'Guardians', 'Guardians of the Galaxy', 'Star-Lord', 'Peter Quill', 'Peter']
Graph RAG context:
The following are knowledge sequence in max depth 2 in the form of `subject predicate, object, predicate_next_hop, object_next_hop ...` extracted based on key entities as subject:
Guardians, is member of, Guardians, was experimented on, by the High Evolutionary
Guardians, is member of, Guardians, considered to tell, origins
Guardians, is member of, Guardians, ori

<b>

Peter Quill is the leader of the Guardians of the Galaxy and the main protagonist of the Guardians of the Galaxy films. He was raised by a group of alien thieves and smugglers, and was abducted from Earth as a child. He is half-human, half-Celestial, and has the ability to wield an energy weapon called the Infinity Stone. He is set to return to the MCU in May 2021.</b>

## 将nl2graphquery作为图RAG中的上下文包含

(Sub)Graph RAG和nl2graphquery的性质是不同的。并不是说其中一个比另一个更好，而是当一个更适合某种类型的问题时。要了解它们之间的区别，可以查看[此演示](https://www.siwei.io/en/demos/graph-rag/)来比较它们。

<video width="938" height="800" 
       src="https://github.com/siwei-io/talks/assets/1651790/05d01e53-d819-4f43-9bf1-75549f7f2be9"  
       controls>
</video>

在实际情况下，我们可能并不总是知道哪种方法更有效，因此，在RAG中最好利用知识图谱的方法是获取检索结果作为上下文，并让LLM + Prompt生成答案，使其都参与其中。

因此，我们可以选择从知识图谱中检索到的两个上下文片段中综合生成答案：
- Graph RAG，默认的检索方法，提取与问题中的关键实体相关的子图。
- NL2GraphQuery，基于查询和知识图谱的模式生成知识图谱查询，默认情况下是关闭状态。

我们可以设置`with_nl2graphquery=True`来启用它，例如：


In [ ]:
graph_rag_retriever_with_nl2graphquery = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
    with_nl2graphquery=True,
)

query_engine_with_nl2graphquery = RetrieverQueryEngine.from_args(
    graph_rag_retriever_with_nl2graphquery,
)

In [ ]:
response = query_engine_with_nl2graphquery.query(
    "What do you know about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))

Graph Store Query:
```
MATCH (p:`entity`)-[:`relationship`]->(m:`entity`) WHERE p.`entity`.`name` == 'Peter Quill'
RETURN m.`entity`.`name`;
```
Graph Store Response:
{'m.entity.name': ['May 2021', 'as a child', 'Guardians of the Galaxy', 'a group of alien thieves and smugglers', 'half-Celestial']}
Entities processed: ['Star', 'Lord', 'Marvel', 'Quill', 'Galaxy', 'Guardians', 'Guardians of the Galaxy', 'Star-Lord', 'Peter Quill', 'Peter']
Entities processed: ['Star', 'Lord', 'Marvel', 'Quill', 'Galaxy', 'Guardians', 'Guardians of the Galaxy', 'Star-Lord', 'Peter Quill', 'Peter']
Graph RAG context:
The following are knowledge sequence in max depth 2 in the form of `subject predicate, object, predicate_next_hop, object_next_hop ...` extracted based on key entities as subject:
Guardians, is member of, Guardians, was experimented on, by the High Evolutionary
Guardians, is member of, Guardians, considered to tell, origins
Guardians, is member of, Guardians, origins, team-up movie
Guardians,

<b>

Peter Quill is the leader of the Guardians of the Galaxy and was abducted from Earth as a child. He is half-human and half-Celestial, and was raised by a group of alien thieves and smugglers. He would return to the MCU in May 2021.</b>

让我们检查响应的元数据，以了解通过检查`response.metadata`检索Graph RAG与nl2graphquery的更多细节。

- **text2Cypher**，它生成一个Cypher查询，以回答上下文。

```cypher
Graph Store Query: MATCH (e:`entity`)-[r:`relationship`]->(e2:`entity`)
WHERE e.`entity`.`name` == 'Peter Quill'
RETURN e2.`entity`.`name`
```
- **SubGraph RAG**，它获取'Peter Quill'的子图以构建上下文。

- 最后，它结合了上下文的两个节点，以综合回答。


In [ ]:
import pprint

pp = pprint.PrettyPrinter()
pp.pprint(response.metadata)

{'46faf6d6-8a71-44c8-ae81-794e71a62fbc': {'graph_schema': 'Node properties: '
                                                          "[{'tag': 'entity', "
                                                          "'properties': "
                                                          "[('name', "
                                                          "'string')]}]\n"
                                                          'Edge properties: '
                                                          "[{'edge': "
                                                          "'relationship', "
                                                          "'properties': "
                                                          "[('relationship', "
                                                          "'string')]}]\n"
                                                          'Relationships: '
                                                          "['(:entity)-[:relationship]->(: